In [1]:
%load_ext autoreload
%autoreload 2

In [35]:
from lib.clue import Clue

game = Clue()
game.play()

Player 1's Hand: {'Kitchen', 'Mr. Green', 'Professor Plum', 'Blackmail'}
Player 2's Hand: {'Wrench', '03:00 AM', 'Knife', 'Dining Room'}
Player 3's Hand: {'Mrs. White', '09:00 PM', 'Candlestick', 'Revenge'}
Player 4's Hand: {'Conservatory', 'Mrs. Peacock', 'Colonel Mustard', 'Cover-up'}
Player 5's Hand: {'Greed', '12:00 AM', '10:00 PM', 'Jealousy', '02:00 AM'}
Player 6's Hand: {'Lead Pipe', '01:00 AM', 'Rope', 'Hall', 'Ballroom'}
Solution: {'suspect': 'Miss Scarlet', 'weapon': 'Revolver', 'room': 'Lounge', 'motive': 'Power', 'time': '11:00 PM'}
Player                   1  2  3  4  5  6
Element Card                             
Suspect Miss Scarlet     ✗  ✗  ✗  ✗  ✗  ✗
        Mr. Green        ✓  ✗  ✗  ✗  ✗  ✗
        Mrs. White       ✗  ✗  ✓  ✗  ✗  ✗
        Mrs. Peacock     ✗  ✗  ✗  ✓  ✗  ✗
        Colonel Mustard  ✗  ✗  ✗  ✓  ✗  ✗
        Professor Plum   ✓  ✗  ✗  ✗  ✗  ✗
Weapon  Candlestick      ✗  ✗  ✓  ✗  ✗  ✗
        Knife            ✗  ✓  ✗  ✗  ✗  ✗
        Lead Pipe        ✗  ✗

In [44]:
import numpy as np
from typing import Optional, Union

player = 2
grid = np.full((len(game.index), game.num_players + 1), np.nan)
last_grid = grid.copy()
constraints: list[
    tuple[
        Union[np.ndarray, tuple[np.ndarray, ...]],
        Union[int, tuple[Optional[int], Optional[int]]],
    ]
] = []

# Each card may only be in one place
for i in range(len(game.index)):
    constraints.append((grid[i], 1))

# Each player has game.hands[i] cards
for i, hand in enumerate(game.hands):
    constraints.append((grid[:, i], len(hand)))

# The solution must have exactly one card from each game element
start = 0
for cards in game.elements.values():
    end = start + len(cards)
    constraints.append((grid[start:end, game.num_players], 1))
    start = end

# Fill in the grid with the known cards
for card in game.hands[player]:
    grid[game.index[card], player] = 1

one_of: dict[int, list[set[int]]] = {i: [] for i in range(game.num_players)}

# Fill in the grid with the known cards from previous turns
for suggesting_player, (suggestion, responses) in enumerate(game.history):
    suggesting_player %= game.num_players
    for responding_player, card in responses.items():
        if card is not None:
            if player == suggesting_player:
                grid[game.index[card], responding_player] = 1
            elif player != responding_player:
                indices = [game.index[c] for c in suggestion]
                # At least one of the suggested cards is in the responding player's hand
                constraints.append(
                    (
                        tuple(grid[i : i + 1, responding_player] for i in indices),
                        (1, len(suggestion)),
                    )
                )
                # And no more than len(game.hands[responding_player]) - 1 of the unsuggested cards are in the responding player's hand
                constraints.append(
                    (
                        tuple(
                            grid[i + 1 : j, responding_player]
                            for i, j in zip([-1] + indices, indices + [len(game.index)])
                        ),
                        (0, len(game.hands[responding_player]) - 1),
                    )
                )
                for previous_indices in one_of[responding_player]:
                    if previous_indices.isdisjoint(indices):
                        union = sorted(previous_indices.union(indices))
                        constraints.append(
                            (
                                tuple(
                                    grid[i + 1 : j, responding_player]
                                    for i, j in zip(
                                        [-1] + union, union + [len(game.index)]
                                    )
                                ),
                                (0, len(game.hands[responding_player]) - len(union) // len(indices)),
                            )
                        )
                        one_of[responding_player].append(set(union))
                one_of[responding_player].append(set(indices))
                print(
                    f"Player {responding_player + 1} has at least one of {suggestion}"
                )
        else:
            for card in suggestion:
                grid[game.index[card], responding_player] = 0

while not np.array_equal(grid, last_grid, equal_nan=True):
    last_grid = grid.copy()
    for views, bounds in constraints:
        if not isinstance(views, tuple):
            views = (views,)
        if isinstance(bounds, int):
            lower_bound = upper_bound = bounds
        else:
            lower_bound, upper_bound = bounds
        values = np.concatenate([view.flatten() for view in views])
        if np.sum(np.nan_to_num(values, nan=1)) == lower_bound:
            for view in views:
                view[np.isnan(view)] = 1
        if np.nansum(values) == upper_bound:
            for view in views:
                view[np.isnan(view)] = 0

game.print_grid(grid[:, :-1])

Player 2 has at least one of ['Mr. Green', 'Wrench', 'Lounge']
Player 4 has at least one of ['Colonel Mustard', 'Knife', 'Kitchen']
Player 1 has at least one of ['Miss Scarlet', 'Revolver', 'Ballroom']
Player 2 has at least one of ['Colonel Mustard', 'Knife', 'Kitchen']
Player 2 has at least one of ['Mr. Green', 'Knife', 'Hall']
Player 2 has at least one of ['Professor Plum', 'Lead Pipe', 'Ballroom']
Player 4 has at least one of ['Mr. Green', 'Knife', 'Dining Room']
Player                   1  2  3  4
Element Card                       
Suspect Miss Scarlet        ✗  ✗   
        Mr. Green        ✗     ✗   
        Mrs. White       ✗  ✗  ✓  ✗
        Mrs. Peacock     ✓  ✗  ✗  ✗
        Colonel Mustard        ✗   
        Professor Plum         ✗   
Weapon  Candlestick      ✗  ✗  ✓  ✗
        Knife            ✗     ✗   
        Lead Pipe              ✗   
        Revolver         ✗  ✗  ✓  ✗
        Rope                ✗  ✗   
        Wrench                 ✗   
Room    Hall             

In [51]:
len(constraints), len(one_of)

(44, 4)

In [42]:
for i, hand in enumerate(game.hands):
    print(f"Player {i + 1} has {len(hand)} cards")

Player 1 has 3 cards
Player 2 has 4 cards
Player 3 has 4 cards
Player 4 has 4 cards


In [52]:
from lib.clue import SimpleSolver

game.print_grid(SimpleSolver().grid(game, player))

Player                   1  2  3  4
Element Card                       
Suspect Miss Scarlet        ✗  ✗   
        Mr. Green        ✗     ✗   
        Mrs. White       ✗  ✗  ✓  ✗
        Mrs. Peacock     ✓  ✗  ✗  ✗
        Colonel Mustard        ✗   
        Professor Plum         ✗   
Weapon  Candlestick      ✗  ✗  ✓  ✗
        Knife            ✗     ✗   
        Lead Pipe              ✗   
        Revolver         ✗  ✗  ✓  ✗
        Rope                ✗  ✗   
        Wrench                 ✗   
Room    Hall             ✗  ✓  ✗  ✗
        Lounge                 ✗   
        Dining Room         ✗  ✗  ✗
        Kitchen                ✗   
        Ballroom               ✗   
        Conservatory     ✗  ✗  ✓  ✗


In [39]:
[tuple(indices) for indices in np.argwhere(np.isnan(grid))]

[(np.int64(0), np.int64(3)),
 (np.int64(0), np.int64(4)),
 (np.int64(2), np.int64(1)),
 (np.int64(2), np.int64(2)),
 (np.int64(2), np.int64(3)),
 (np.int64(2), np.int64(4)),
 (np.int64(4), np.int64(1)),
 (np.int64(4), np.int64(3)),
 (np.int64(4), np.int64(4)),
 (np.int64(7), np.int64(3)),
 (np.int64(7), np.int64(4)),
 (np.int64(9), np.int64(2)),
 (np.int64(9), np.int64(4)),
 (np.int64(10), np.int64(3)),
 (np.int64(10), np.int64(4))]

In [37]:
grid[1:5].__getitem__

20

In [24]:
np.isin(grid, grid[1:5])

array([[False, False,  True, False, False],
       [False, False,  True, False, False],
       [ True,  True,  True,  True,  True],
       [False,  True,  True, False, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [False, False,  True, False, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True, False, False],
       [False, False,  True, False, False],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [False,  True,  True, False,  True],
       [False, False,  True, False,  True],
       [False,  True,  True, False,  True],
       [ True,  True,  True,  True,  True]])

In [17]:
one_of

{0: [], 1: [{1, 3, 7}, {1, 2, 3, 5, 7, 8}, {2, 5, 8}], 2: []}

In [ ]:
"""
Player 4's CP-SAT Solver Grid:
Player                   1  2  3  4
Element Card                       
Suspect Miss Scarlet        ✗     ✗
        Mr. Green        ✗  ✗  ✗  ✓
        Mrs. White       ✗  ✗     ✗
        Mrs. Peacock     ✗  ✓  ✗  ✗
        Colonel Mustard        ✗  ✗
        Professor Plum            ✗
Weapon  Candlestick      ✗  ✗     ✗
        Knife            ✗  ✗     ✗
        Lead Pipe        ✓  ✗  ✗  ✗
        Revolver                  ✗
        Rope                   ✗  ✗
        Wrench           ✗  ✗  ✗  ✓
Room    Hall                   ✗  ✗
        Lounge           ✗  ✗  ✗  ✓
        Dining Room      ✗  ✗     ✗
        Kitchen             ✗     ✗
        Ballroom         ✗  ✗  ✗  ✓
        Conservatory           ✗  ✗
"""

In [ ]:
from itertools import chain
import numpy as np
import sympy as sp

grid = np.array(
    [
        [
            sp.Symbol(f"{player} has {card}", integer=True)
            for player in chain(
                (f"Player {player + 1}" for player in range(game.num_players)),
                ["Solution"],
            )
        ]
        for card in game.index
    ]
)

constraints = []

for player in range(len(game.index)):
    constraints.append(grid[player].sum() == 1)

for player in range(game.num_players):
    constraints.append(grid[:, player].sum() == len(game.hands[player]))

for cards in game.elements.values():
    constraints.append(
        grid[[game.index[card] for card in cards], game.num_players].sum() == 1
    )

for player, (suggestion, responses) in enumerate(game.history):
    for j, card in responses.items():
        if card is not None:
            constraints.append(
                grid[[game.index[card] for card in suggestion], j].sum() >= 1
            )
        else:
            constraints.append(
                grid[[game.index[card] for card in suggestion], j].sum() == 0
            )

player = 0

for card in game.hands[player]:
    constraints.append(grid[game.index[card], player] == 1)

for player in range(len(game.index)):
    for j in range(game.num_players):
        symbol = grid[player, j]
        print(sp.solve(constraints + [symbol == 0], symbol))

# sp.linsolve(constraints, *grid.flatten())

In [ ]:
from functools import partial
import kanren as kr

grid = np.array(
    [
        [
            kr.var(f"{player} has {card}")
            for player in chain(
                (f"Player {player + 1}" for player in range(game.num_players)),
                ["Solution"],
            )
        ]
        for card in game.index
    ]
)

kr.run(
    0,
    grid[0, 0],
    kr.lall(*np.vectorize(partial(kr.membero, ls=(0, 1)))(grid).flatten())
)

In [ ]:
np.vectorize(kr.eq)(grid[0], 1)